In [1]:
import os
import sys
import datetime
import subprocess
import numpy as np

import rmgpy.chemkin
import rmgpy.data.kinetics
# import cantera as ct

# sys.path.append('/work/westgroup/harris.se/autoscience/reaction_calculator/dft')
# import thermokinetic_fun


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load the base model
basedir = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week'

new_model_dir = basedir
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(species_list)} species, {len(reaction_list)} reactions')


130 species, 2488 reactions


In [3]:
# Load the new kinetics library
DFT_DIR = "/work/westgroup/harris.se/autoscience/reaction_calculator/dft/"
kinetics_lib = os.path.join(DFT_DIR, 'kinetics', 'kinetics')
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.load_libraries(kinetics_lib)
# print(ark_kinetics_database.libraries)
print(f'{len(ark_kinetics_database.libraries[""].entries)} entries loaded')

48 entries loaded


In [4]:
# load the uncertainty matrix
reaction_uncertainty_file = os.path.join(basedir, 'gao_reaction_uncertainty.npy')
species_uncertainty_file = os.path.join(basedir, 'gao_species_uncertainty.npy')

reaction_uncertainty = np.load(reaction_uncertainty_file)
species_uncertainty = np.load(species_uncertainty_file)

uncertainty_cutoff = 3.0

rxn_change_list = []
for i in range(0, len(reaction_list)):
    if reaction_uncertainty[i] > uncertainty_cutoff:
        rxn_change_list.append(i)

sp_change_list = []
for i in range(0, len(species_list)):
    if species_uncertainty[i] > uncertainty_cutoff:
        sp_change_list.append(i)


In [5]:
species_list[87].smiles

'CC(=O)CCOO'

In [6]:
species_uncertainty[87]

1.5

In [7]:
# stitch each of the library reactions into the mechanism (depends on library index matching indexing in chemkin file)

total_changed = 0
for key in ark_kinetics_database.libraries[''].entries.keys():
    entry = ark_kinetics_database.libraries[''].entries[key]
    index = entry.index
#     print(index, '\t', entry.label, '\t', reaction_list[index])
    for i in range(0, len(reaction_list)):
        if entry.item.is_isomorphic(reaction_list[i]):
            rmg_rxn_index = i
            break
    else:  
        print(f'could not match reaction index {index}')
        continue
  
    
#     # change every possible reaction
#     print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
#     reaction_list[rmg_rxn_index].kinetics = entry.data
    
    if rmg_rxn_index in rxn_change_list:
        print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
        reaction_list[rmg_rxn_index].kinetics = entry.data
        total_changed += 1

#     if rmg_rxn_index in change_list or rmg_rxn_index in change_list_fs:
#         print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
#         reaction_list[rmg_rxn_index].kinetics = entry.data
#         total_changed += 1
print(total_changed, 'changed')

changing reaction 324 C4H8OOH2-4O2(229) <=> C4H8OOH1-3O2(225)
changing reaction 370 CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1)
changing reaction 371 CH3CHO(35) + PC4H9(182) <=> CH2CHO(21) + butane(1)
changing reaction 585 CH2(23) + C3H6(12) <=> CH3(18) + C3H5-A(94)
changing reaction 650 HO2(16) + [CH2]C[CH]C(748) <=> O2(2) + SC4H9(183)
changing reaction 655 HO2(16) + [CH2]C[CH]C(748) <=> O2(2) + PC4H9(182)
changing reaction 739 CH2CHO(21) + C4H8-1(188) <=> CH3CHO(35) + C4H71-3(190)
changing reaction 748 HCO(19) + C4H8-2(189) <=> CH2O(9) + C4H71-3(190)
changing reaction 749 HCO(19) + C4H8-1(188) <=> CH2O(9) + C4H71-3(190)
changing reaction 754 CH2(23) + C4H8-2(189) <=> CH3(18) + C4H71-3(190)
changing reaction 894 C3H6(12) + C4H72-2,4OOH(244) <=> C3H5-A(94) + CC(CCOO)OO(787)
changing reaction 985 HO2(16) + [CH2]C[O](703) <=> O2(2) + C2H5O(49)
changing reaction 1003 HO2(16) + C4H71-4(191) <=> O2(2) + C4H8-1(188)
changing reaction 1026 CH3CHO(35) + C4H71-4(191) <=> CH2CHO(21) + C4H

In [8]:
# Load the old thermo library
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
thermo_lib = os.path.join(DFT_DIR, 'thermo', 'thermo')
ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
ark_thermo_database.load_libraries(thermo_lib)
print(f'{len(ark_thermo_database.libraries["harris_butane"].entries)} entries')

178 entries


In [9]:
# stitch each of the library thermos into the mechanism


sp_changed = 0
for key in ark_thermo_database.libraries['harris_butane'].entries.keys():
    entry = ark_thermo_database.libraries['harris_butane'].entries[key]

    for i, species in enumerate(species_list):
        if entry.item.smiles == species.smiles:
            rmg_species_index = i
            break
    else:
        entry_sp = rmgpy.species.Species(smiles=entry.item.smiles)
        for j, species in enumerate(species_list):
            if entry_sp.is_isomorphic(species.molecule[0]):
                rmg_species_index = i
                break
        else:
#             print(f'could not match species index {entry}')
            continue
            raise ValueError (f'could not match species index {entry}')
        
        
    if rmg_species_index in sp_change_list:
        print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
        species_list[rmg_species_index].thermo = entry.data
        
        sp_changed += 1
    
print(sp_changed, 'changed')

Changing estimated thermo 92: [CH2]C[O](703)
Changing estimated thermo 93: [CH2]C[CH]C(748)
Changing estimated thermo 94: [CH2]CC(C)[O](766)
Changing estimated thermo 99: [CH2]C(=O)COO(911)
Changing estimated thermo 102: [O]OCC(=O)COO(928)
Changing estimated thermo 103: [O]C[O](988)
Changing estimated thermo 106: [CH2]C=C[CH2](2534)
Changing estimated thermo 110: C=CC(C)O[O](4243)
Changing estimated thermo 111: CC=CCO[O](4244)
Changing estimated thermo 112: [CH2]C1CC1(4343)
Changing estimated thermo 113: C=CCCO[O](4407)
Changing estimated thermo 89: [CH2]C[CH2](688)
Changing estimated thermo 120: [CH2]C(O)C=O(5659)
Changing estimated thermo 126: [O]OCC(O)C=O(11705)
Changing estimated thermo 127: O=[C]C(O)COO(11851)
15 changed


In [10]:
# custom save
chemkin_file = os.path.join('/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week', f'cutoff3_20230501-2.inp')
# chemkin_file = os.path.join('/home/moon/autoscience/autoscience/butane/models/modifications', f'cutoff3_20230202.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={chemkin_file[:-4]}.cti'])



Wrote CTI mechanism file to '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/cutoff3_20230501-2.cti'.
Mechanism contains 130 species and 2523 reactions.
Validating mechanism...PASSED.


CompletedProcess(args=['ck2cti', '--input=/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/cutoff3_20230501-2.inp', '--transport=/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/tran.dat', '--output=/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/cutoff3_20230501-2.cti'], returncode=0)